# Lab7-1 Learning rate, Evaluation

- 목표
  - 별도의 테스트 데이터로 모델을 평가

## Training and Test datasets

- training과 테스트 데이터를 분리
  - training data는 학습에만 사용
  - test data는 평가에만 사용

### Example

In [5]:
# traing datasets
x_data = [
    [1, 2, 1], [1, 3, 2], [1, 3, 4], [1, 5, 5],
    [1, 7, 5], [1, 2, 5], [1, 6, 6], [1, 7, 7]
]

y_data = [
    [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0],
    [0, 1, 0], [1, 0, 0], [1, 0, 0]
]

In [8]:
# test dataset for evaluation of model
# 학습이 종료된 이후에 평가에 이용
x_test=[[2, 1, 1], [3, 1, 2], [3, 3, 4]]
y_test=[[0, 0, 1], [0, 0, 1], [0, 0, 1]]

## lab 01

In [12]:
import tensorflow as tf

x_data = [
    [1, 2, 1], [1, 3, 2], [1, 3, 4], [1, 5, 5],
    [1, 7, 5], [1, 2, 5], [1, 6, 6], [1, 7, 7]
]

y_data = [
    [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0],
    [0, 1, 0], [1, 0, 0], [1, 0, 0]
]

x_test=[[2, 1, 1], [3, 1, 2], [3, 3, 4]]
y_test=[[0, 0, 1], [0, 0, 1], [0, 0, 1]]

In [25]:
X = tf.placeholder(tf.float32, [None, 3])
Y = tf.placeholder(tf.float32, [None, 3])

W = tf.Variable(tf.random_normal([3, 3]), name='weight')
b = tf.Variable(tf.random_normal([3]), name='bias')

In [26]:
hypothesis = tf.nn.softmax(tf.matmul(X, W)+b)
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [28]:
prediction = tf.arg_max(hypothesis, 1)
is_correct = tf.equal(prediction, tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [31]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(201):
        cost_val, W_val, _ =sess.run([cost, W, optimizer], 
                                    feed_dict={X:x_data, Y:y_data})
        if step % 40 == 0 :
            print(step, cost_val, W_val)
            
    print("prediction:", sess.run(prediction, feed_dict={X:x_test}))
    print("accuracy:", sess.run(accuracy, feed_dict={X:x_test, Y:y_test}))

0 2.07086 [[ 0.1499214   0.20344086  1.0874815 ]
 [-0.01043166 -0.90407634  1.12298834]
 [ 0.72130489  1.52626073 -0.90227354]]
40 0.862651 [[-0.05435106  0.07402195  1.4211731 ]
 [-0.01841847 -0.44451308  0.67141187]
 [ 0.87389344  1.0583415  -0.58694232]]
80 0.715662 [[-0.26904586 -0.00619927  1.71608901]
 [ 0.04245763 -0.18347742  0.34950021]
 [ 0.87221152  0.81784707 -0.34476626]]
120 0.642556 [[-0.46703184 -0.05025299  1.95812893]
 [ 0.1201399  -0.06083836  0.14917898]
 [ 0.86404997  0.71406865 -0.23282647]]
160 0.601785 [[-0.64355004 -0.07187367  2.15626812]
 [ 0.17005198 -0.00689072  0.04531944]
 [ 0.88229197  0.67569005 -0.21268956]]
200 0.573144 [[-0.80323005 -0.08019599  2.32427096]
 [ 0.19664179  0.01841658 -0.00657754]
 [ 0.91848636  0.66113317 -0.2343272 ]]
prediction: [2 2 2]
accuracy: 1.0


## learn rate: NaN

- 부적당한 Learning rate이 문제점

![](./images/19/lr.jpg)

- Learning rate이 너무 크면: cost함수가 발산
- Learning rate가 너무 작으면: cost함수가 local minimum에 빠질 가능성 높음

### Learning rate이 너무 끌 경우

In [59]:
import tensorflow as tf

x_data = [
    [1, 2, 1], [1, 3, 2], [1, 3, 4], [1, 5, 5],
    [1, 7, 5], [1, 2, 5], [1, 6, 6], [1, 7, 7]
]

y_data = [
    [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0],
    [0, 1, 0], [1, 0, 0], [1, 0, 0]
]

x_test=[[2, 1, 1], [3, 1, 2], [3, 3, 4]]
y_test=[[0, 0, 1], [0, 0, 1], [0, 0, 1]]

X = tf.placeholder(dtype=tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, [None, 3])

W = tf.Variable(tf.random_normal([3, 3]), name='weight')
b = tf.Variable(tf.random_normal([3]), name='bias')

hypothesis = tf.nn.softmax(tf.matmul(X, W)+b)
cost = tf.reduce_mean(tf.reduce_sum(Y*-tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.5).minimize(cost)

prediction = tf.arg_max(hypothesis, 1)
is_correct = tf.equal(prediction, tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(51):
        cost_val, W_val, _ = sess.run([cost, W, optimizer], feed_dict={X:x_data, Y:y_data})
        if step % 10 == 0:
            print(step, cost_val, W_val)
    
    print("prediction:", sess.run(prediction, feed_dict={X:x_test}))
    print("accuracy:", sess.run(accuracy, feed_dict={X:x_test, Y:y_test}))

0 3.17779 [[-1.23723269 -0.67176002  0.65561879]
 [ 1.57405329  3.50201225 -5.04815626]
 [ 2.15199924  0.59882474 -5.39827728]]
10 nan [[ nan  nan  nan]
 [ nan  nan  nan]
 [ nan  nan  nan]]
20 nan [[ nan  nan  nan]
 [ nan  nan  nan]
 [ nan  nan  nan]]
30 nan [[ nan  nan  nan]
 [ nan  nan  nan]
 [ nan  nan  nan]]
40 nan [[ nan  nan  nan]
 [ nan  nan  nan]
 [ nan  nan  nan]]
50 nan [[ nan  nan  nan]
 [ nan  nan  nan]
 [ nan  nan  nan]]
prediction: [0 0 0]
accuracy: 0.0


- Nan: cost의 발산

### Learning rate가 너무 작을 때

In [60]:
import tensorflow as tf

x_data = [
    [1, 2, 1], [1, 3, 2], [1, 3, 4], [1, 5, 5],
    [1, 7, 5], [1, 2, 5], [1, 6, 6], [1, 7, 7]
]

y_data = [
    [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0],
    [0, 1, 0], [1, 0, 0], [1, 0, 0]
]

x_test=[[2, 1, 1], [3, 1, 2], [3, 3, 4]]
y_test=[[0, 0, 1], [0, 0, 1], [0, 0, 1]]

X = tf.placeholder(dtype=tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, [None, 3])

W = tf.Variable(tf.random_normal([3, 3]), name='weight')
b = tf.Variable(tf.random_normal([3]), name='bias')

hypothesis = tf.nn.softmax(tf.matmul(X, W)+b)
cost = tf.reduce_mean(tf.reduce_sum(Y*-tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-10).minimize(cost)

prediction = tf.arg_max(hypothesis, 1)
is_correct = tf.equal(prediction, tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(101):
        cost_val, W_val, _ = sess.run([cost, W, optimizer], feed_dict={X:x_data, Y:y_data})
        if step % 10 == 0:
            print(step, cost_val, W_val)
    
    print("prediction:", sess.run(prediction, feed_dict={X:x_test}))
    print("accuracy:", sess.run(accuracy, feed_dict={X:x_test, Y:y_test}))

0 4.7365 [[ 0.13072924 -0.10709396 -2.06912446]
 [ 1.01333606 -0.28704023 -1.06621075]
 [-0.34327912  1.5086596  -0.56471342]]
10 4.7365 [[ 0.13072924 -0.10709396 -2.06912446]
 [ 1.01333606 -0.28704023 -1.06621075]
 [-0.34327912  1.5086596  -0.56471342]]
20 4.7365 [[ 0.13072924 -0.10709396 -2.06912446]
 [ 1.01333606 -0.28704023 -1.06621075]
 [-0.34327912  1.5086596  -0.56471342]]
30 4.7365 [[ 0.13072924 -0.10709396 -2.06912446]
 [ 1.01333606 -0.28704023 -1.06621075]
 [-0.34327912  1.5086596  -0.56471342]]
40 4.7365 [[ 0.13072924 -0.10709396 -2.06912446]
 [ 1.01333606 -0.28704023 -1.06621075]
 [-0.34327912  1.5086596  -0.56471342]]
50 4.7365 [[ 0.13072924 -0.10709396 -2.06912446]
 [ 1.01333606 -0.28704023 -1.06621075]
 [-0.34327912  1.5086596  -0.56471342]]
60 4.7365 [[ 0.13072924 -0.10709396 -2.06912446]
 [ 1.01333606 -0.28704023 -1.06621075]
 [-0.34327912  1.5086596  -0.56471342]]
70 4.7365 [[ 0.13072924 -0.10709396 -2.06912446]
 [ 1.01333606 -0.28704023 -1.06621075]
 [-0.34327912  1.

## Non-normalized input

![](./images/20/nonnormal.jpg)

- feature의 값차이가 커서 발생하는 발산

![](./images/20/nonnormal2.jpg)

## 해결 방법

- xy(MinMaxScaler(xy))


- 데이터가 크거나 데이터의 scale이 다를 경우에는 nomalize를 하는 것이 추천됨

![](./images/20/nonnormal3.jpg)

### Test after nomalized data

![](./images/20/nonnormal4.jpg)